In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import glob, os

folder_path = 'D:\VSCode\Lead Analysis'
csv_files = glob.glob(os.path.join(folder_path, '**', '*.csv'), recursive=True)


In [ ]:
# My counter without different encoding methods
leads_breakdown = pd.DataFrame(columns=['Filename', '# of rows', 'Column List'])
print(len(csv_files))
for file in csv_files:
    name = Path(file).name
    print(name)
    row_count = 0
    columns_df = pd.read_csv(file, nrows=0)
    columns = columns_df.columns
    for chunk in pd.read_csv(file,chunksize=1000000 ,usecols=[0], on_bad_lines='skip'):
        row_count += len(chunk)
    column_list = ', '.join(columns)
    new_row = pd.DataFrame([{'Filename': name, '# of rows': row_count, 'Column List': column_list}])
    leads_breakdown = pd.concat([leads_breakdown, new_row], ignore_index=True)
    print(f'\nDone: {row_count}\n')


In [ ]:
# Initialize the DataFrame
leads_breakdown = pd.DataFrame(columns=['Filename', '# of rows', 'Column List'])

# List of possible encodings to try
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']

print(len(csv_files))
for file in csv_files:
    # Get the filename
    name = Path(file).name
    
    print(name)
    row_count = 0
    columns = None
    
    # Try different encodings
    for encoding in encodings:
        try:
            columns_df = pd.read_csv(file, nrows=0, encoding=encoding)
            columns = columns_df.columns
            break  # If it succeeds, no need to try other encodings
        except UnicodeDecodeError:
            continue  # Try the next encoding

    if columns is None:
        print(f"Failed to decode {name} with available encodings.")
        continue
    
    # Count rows in chunks
    for chunk in pd.read_csv(file, chunksize=1000000, usecols=[0], on_bad_lines='skip', encoding=encoding):
        row_count += len(chunk)
    
    # Create new row with information
    column_list = ', '.join(columns)
    new_row = pd.DataFrame([{'Filename': name, '# of rows': row_count, 'Column List': column_list}])
    leads_breakdown = pd.concat([leads_breakdown, new_row], ignore_index=True)
    print(f'\nDone: {row_count}\n')


In [ ]:
encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
keyword = 'plumb'

chunk_division = range(15)
temp_files = list(range(500))
csv_chunk_size = 1000000
filtered_chunks = []
folder_path = 'D:\VSCode\Lead Analysis'
csv_files = glob.glob(os.path.join(folder_path, '**', '*.csv'), recursive=True)

def chunk_files(file_list): # Equally divides the list based off of chunk size
    files_per_chunk = int(len(file_list) / len(chunk_division))
    chunk = {}
    x = 0
    for i in chunk_division:
        y = x + files_per_chunk if i < max(chunk_division) else len(file_list)
        chunk[f'{i}'] = file_list[x:y]
        x = y
    return chunk

def chunk_search(file_list:dict, chunk_start, chunk_end):
    search_results = pd.DataFrame()
    processing_list = {}

    # inclusive range of file index
    if chunk_start == chunk_end:
        processing_list = file_list
    else:
        processing_range = range(chunk_start, chunk_end + 1)
        for i in processing_range:
            print(i)
            processing_list[f'{i}'] = file_list[f'{i}']
    print(file_list)
    print(type(processing_list))


    for index, files in processing_list.items():
        for file in files:
            print(f'Current File: {file}')
            for encoding in encodings:
                try:
                    for chunk in pd.read_csv(file, chunksize=1000000, on_bad_lines='skip', encoding=encoding):
                        result = chunk[chunk.apply(lambda row: row.astype(str).str.contains('plumb').any(), axis=1)]
                        search_results = pd.concat([search_results, result], ignore_index=True)
                    break  # If it succeeds, no need to try other encodings
                except UnicodeDecodeError:
                    continue  # Try the next encoding

    search_results.to_csv('Output\plumb_list.csv', index=False) #16.49 to search whole dataset

greg = {}
greg[1] = csv_files
print(greg)
chunk_search(greg, 1, 1) # inclusive

In [29]:
leads_breakdown.to_csv('Output\lead_analysis.csv', index=False)
leads_breakdown.to_csv('Output\lead_plumb.csv', index=False)

In [ ]:
# 2063 prev set                                                                                                                            Click execute above cells ^